In [14]:
import numpy as np


In [2]:
from sklearn.metrics import roc_auc_score

In [39]:
import numpy as np
source1 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca.txt"
source2 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca-cls.txt"
source3 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd.txt"
source4 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd-cls.txt"
source5 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c5.txt"
source6 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c10.txt"
source7 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d5.txt"
source8 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d10.txt"
source9 = "F:/MSc DS/Project/dms/trained_dms_txt/word2dm.txt"

# dim = 17 #

def get_dict_from_source(source = source5,dim =17):
    if source ==source1 or source==source3:
        dim = 15
    whole =[]

    with open(source) as f:
        for line in f:
            linelist = line.split()
            for a in linelist:
                whole.append(a)
    chunk = dim**2+1
    n = len(whole)/chunk
    voc = []
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        a = whole[i * chunk + 1:(i + 1) * chunk]
        a = np.array(a, dtype='float')
        a.resize([dim, dim])
        dic[voc[i]] = a

    return dic

def kba(a, b):
    m = b-a
    eig,_ = np.linalg.eig(m)
    Kba = sum(eig)/sum(abs(eig))
    return Kba

def halfmatrix(m):
    eig,vec = np.linalg.eig(m)
    n = len(eig)
    for i in range(n):
        if eig[i]<0:
            eig[i]=0
    eignew = eig**0.5
    dim = m.shape[0]
    e = np.eye(dim)
    for i in range(dim):
        e[i][i] =eignew[i]
    vec_T = np.linalg.inv(vec)
    half = vec@e@vec_T
    return half

def compute_Kba(listA,listB, dic, method ='add'):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get new dm using composition method
    compute Ke pred for each pair
    if oov set Ke to oov  # after test all in voc

    composition method
    add or pointwise
    '''

    n = len(listA)
    if method == 'add':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] + dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] + dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))

    if method == 'pointwise':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] * dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] * dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))
            
    if method == 'dot':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()]@dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()]@dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))
    
    if method == 'mult':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            n1 = dic[phraseA[0].lower()]
            v1 = dic[phraseA[1].lower()]
            n2 = dic[phraseB[0].lower()]
            v2 = dic[phraseB[1].lower()]
            newA = halfmatrix(n1)*v1*halfmatrix(n1)
            newB = halfmatrix(n2)*v2*halfmatrix(n2)
            Kba_list.append(kba(newA, newB))
            
    return Kba_list






# test different source with different composition methods

# get sv dataset
# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-SV.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])

# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val

In [40]:
dic1 = get_dict_from_source(source1)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5185185185185185


In [41]:
dic1 = get_dict_from_source(source2)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6459259259259259


In [42]:
dic1 = get_dict_from_source(source3)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5788203017832648


In [43]:
dic1 = get_dict_from_source(source4)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6013717421124829


In [44]:
dic1 = get_dict_from_source(source5)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5788203017832648


In [45]:
dic1 = get_dict_from_source(source6)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5481481481481482


In [46]:
dic1 = get_dict_from_source(source7)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.4


In [47]:
dic1 = get_dict_from_source(source8)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.44460905349794244


In [48]:
dic1 = get_dict_from_source(source9)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.42562414266117965
